In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re
import time
import gc
import warnings
import pickle
import os

In [2]:
import torch.nn.functional as F
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModelForCausalLM
)
warnings.filterwarnings('ignore')

In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [4]:
class InferenceConfig:
    """Configuration for inference phase"""
    
    # Check if GPU is available
    HAS_GPU = torch.cuda.is_available()
    DEVICE = torch.device('cuda' if HAS_GPU else 'cpu')
    
    # Model directory
    MODEL_SAVE_DIR = "/kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/"
    
    # Inference parameters
    BATCH_SIZE = 16 if HAS_GPU else 8
    MAX_LENGTH = 512
    DATALOADER_WORKERS = 4 if HAS_GPU else 0

In [5]:
class InferenceTextDataset(Dataset):
    """Dataset for inference"""
    
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        try:
            text = str(self.texts[idx])
            text = re.sub(r'\s+', ' ', text.strip())
            
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
            
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten()
            }
        except Exception as e:
            print(f"Error processing sample {idx}: {e}")
            return {
                'input_ids': torch.zeros(self.max_length, dtype=torch.long),
                'attention_mask': torch.zeros(self.max_length, dtype=torch.long)
            }

In [6]:
class DeBERTaInference:
    """Load and run inference with trained DeBERTa model"""
    
    def __init__(self, model_path):
        self.model_path = model_path
        self.tokenizer = None
        self.model = None
        self.config = None
        
    def load_model(self):
        """Load the saved DeBERTa model"""
        try:
            print(f"Loading DeBERTa model from {self.model_path}")
            
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            
            # Load model
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
            self.model = self.model.to(InferenceConfig.DEVICE)
            self.model.eval()
            
            # Load training info if available
            info_path = os.path.join(self.model_path, 'training_info.pkl')
            if os.path.exists(info_path):
                with open(info_path, 'rb') as f:
                    self.config = pickle.load(f)
                print(f"Model trained for {self.config.get('config', {}).get('epochs', 'unknown')} epochs")
            
            print("DeBERTa model loaded successfully")
            
        except Exception as e:
            print(f"Error loading DeBERTa model: {e}")
            raise
    
    def predict(self, texts):
        """Make predictions on new texts"""
        if self.model is None:
            self.load_model()
        
        print(f"Making DeBERTa predictions on {len(texts)} samples...")
        
        try:
            predictions = []
            dataset = InferenceTextDataset(texts, self.tokenizer, InferenceConfig.MAX_LENGTH)
            dataloader = DataLoader(
                dataset, 
                batch_size=InferenceConfig.BATCH_SIZE, 
                shuffle=False, 
                num_workers=InferenceConfig.DATALOADER_WORKERS
            )
            
            with torch.no_grad():
                for batch in dataloader:
                    input_ids = batch['input_ids'].to(InferenceConfig.DEVICE)
                    attention_mask = batch['attention_mask'].to(InferenceConfig.DEVICE)
                    
                    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                    probs = F.softmax(outputs.logits, dim=-1)
                    predictions.extend(probs[:, 1].cpu().numpy())
            
            return np.array(predictions[:len(texts)])
            
        except Exception as e:
            print(f"Error during DeBERTa prediction: {e}")
            return np.full(len(texts), 0.5)

In [7]:
class ZeroShotInference:
    """Load and run inference with trained zero-shot model"""
    
    def __init__(self, model_path):
        self.model_path = model_path
        self.tokenizer = None
        self.model = None
        self.cache = {}
        self.config = None
        
    def load_model(self):
        """Load the saved zero-shot model"""
        try:
            print(f"Loading zero-shot model from {self.model_path}")
            
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            
            # Load model
            self.model = AutoModelForCausalLM.from_pretrained(self.model_path)
            self.model = self.model.to(InferenceConfig.DEVICE)
            self.model.eval()
            
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # Load cache and config
            info_path = os.path.join(self.model_path, 'zero_shot_info.pkl')
            if os.path.exists(info_path):
                with open(info_path, 'rb') as f:
                    data = pickle.load(f)
                    self.cache = data.get('cache', {})
                    self.config = data.get('config', {})
            
            print("Zero-shot model loaded successfully")
            
        except Exception as e:
            print(f"Error loading zero-shot model: {e}")
            raise
    
    def get_log_likelihood(self, text):
        """Calculate log-likelihood with caching"""
        if text in self.cache:
            return self.cache[text]
        
        try:
            if len(text.split()) > 150:
                text = ' '.join(text.split()[:150])
            
            inputs = self.tokenizer(
                text,
                return_tensors='pt',
                truncation=True,
                max_length=InferenceConfig.MAX_LENGTH,
                padding=True
            ).to(InferenceConfig.DEVICE)
            
            with torch.no_grad():
                outputs = self.model(**inputs, labels=inputs['input_ids'])
                log_likelihood = -outputs.loss.item()
            
            self.cache[text] = log_likelihood
            return log_likelihood
            
        except Exception as e:
            return 0.0
    
    def perturb_text(self, text, num_perturbations=10):
        """Simple text perturbation for inference"""
        perturbations = []
        tokens = text.split()
        
        if len(tokens) < 3:
            return [text]
        
        for _ in range(num_perturbations):
            try:
                perturbed_tokens = tokens.copy()
                
                # Random word dropout
                if len(perturbed_tokens) > 3 and np.random.random() > 0.5:
                    drop_idx = np.random.randint(0, len(perturbed_tokens))
                    perturbed_tokens.pop(drop_idx)
                
                perturbed_text = ' '.join(perturbed_tokens)
                if perturbed_text != text and len(perturbed_text.strip()) > 0:
                    perturbations.append(perturbed_text)
                    
            except Exception:
                continue
        
        return perturbations if perturbations else [text]
    
    def predict(self, texts):
        """Make zero-shot predictions"""
        if self.model is None:
            self.load_model()
        
        print(f"Making zero-shot predictions on {len(texts)} samples...")
        
        predictions = []
        
        for i, text in enumerate(texts):
            if i % 50 == 0:
                print(f"Processing text {i+1}/{len(texts)}")
            
            try:
                # Original log-likelihood
                original_ll = self.get_log_likelihood(text)
                
                # Perturbed log-likelihoods
                perturbations = self.perturb_text(text)
                perturbed_lls = [self.get_log_likelihood(p) for p in perturbations]
                
                if perturbed_lls:
                    mean_perturbed_ll = np.mean(perturbed_lls)
                    std_perturbed_ll = np.std(perturbed_lls) if len(perturbed_lls) > 1 else 1.0
                    
                    curvature_score = (original_ll - mean_perturbed_ll) / (std_perturbed_ll + 1e-8)
                    prob = 1 / (1 + np.exp(-curvature_score * 2))
                    predictions.append(prob)
                else:
                    predictions.append(0.5)
                    
            except Exception as e:
                print(f"Error processing text {i}: {e}")
                predictions.append(0.5)
        
        return np.array(predictions)

In [8]:
class TFIDFInference:
    """Load and run inference with trained TF-IDF model"""
    
    def __init__(self, model_path):
        self.model_path = model_path
        self.vectorizers = None
        self.classifiers = None
        self.feature_extractor = None
        self.is_loaded = False
        
    def load_model(self):
        """Load the saved TF-IDF model"""
        try:
            print(f"Loading TF-IDF model from {self.model_path}")
            
            model_file = os.path.join(self.model_path, 'tfidf_model.pkl')
            with open(model_file, 'rb') as f:
                data = pickle.load(f)
                
            self.vectorizers = data['vectorizers']
            self.classifiers = data['classifiers']
            self.feature_extractor = data['feature_extractor']
            self.is_loaded = data['is_fitted']
            
            print("TF-IDF model loaded successfully")
            
        except Exception as e:
            print(f"Error loading TF-IDF model: {e}")
            raise
    
    def predict(self, texts):
        """Make TF-IDF predictions"""
        if not self.is_loaded:
            self.load_model()
        
        print(f"Making TF-IDF predictions on {len(texts)} samples...")
        
        try:
            # Extract features
            all_features = []
            for vectorizer in self.vectorizers:
                features = vectorizer.transform(texts)
                all_features.append(features)
            
            stat_features = self.feature_extractor.extract_features(texts)
            
            # Get predictions from all classifiers
            all_predictions = []
            for i, classifier in enumerate(self.classifiers):
                if i < len(all_features):
                    combined_features = np.hstack([
                        all_features[i].toarray(),
                        stat_features.values
                    ])
                else:
                    combined_tfidf = np.hstack([f.toarray() for f in all_features])
                    combined_features = np.hstack([
                        combined_tfidf,
                        stat_features.values
                    ])
                
                if hasattr(classifier, 'predict_proba'):
                    pred = classifier.predict_proba(combined_features)[:, 1]
                else:
                    pred = classifier.predict(combined_features).astype(float)
                
                all_predictions.append(pred)
            
            # Ensemble prediction
            weights = [0.4, 0.3, 0.3]
            final_prediction = np.average(all_predictions, axis=0, weights=weights)
            
            return final_prediction
            
        except Exception as e:
            print(f"Error in TF-IDF prediction: {e}")
            return np.full(len(texts), 0.5)

In [9]:
class EnsembleInference:
    """Load and run inference with trained ensemble model"""
    
    def __init__(self, model_path):
        self.model_path = model_path
        self.meta_classifiers = None
        self.is_loaded = False
        
    def load_model(self):
        """Load the saved ensemble model"""
        try:
            print(f"Loading ensemble model from {self.model_path}")
            
            model_file = os.path.join(self.model_path, 'ensemble_model.pkl')
            with open(model_file, 'rb') as f:
                data = pickle.load(f)
                
            self.meta_classifiers = data['meta_classifiers']
            self.is_loaded = data['is_fitted']
            
            print("Ensemble model loaded successfully")
            
        except Exception as e:
            print(f"Error loading ensemble model: {e}")
            raise
    
    def create_advanced_features(self, component_predictions):
        """Create advanced features from component predictions"""
        features = []
        
        # Original predictions
        features.extend(component_predictions)
        
        # Interaction features
        for i in range(len(component_predictions)):
            for j in range(i+1, len(component_predictions)):
                features.append(component_predictions[i] * component_predictions[j])
        
        # Statistical features
        stacked = np.column_stack(component_predictions)
        features.append(np.mean(stacked, axis=1))
        features.append(np.std(stacked, axis=1))
        features.append(np.max(stacked, axis=1))
        features.append(np.min(stacked, axis=1))
        
        # Confidence features
        for pred in component_predictions:
            confidence = np.abs(pred - 0.5)
            features.append(confidence)
        
        return np.column_stack(features)
    
    def predict(self, component_predictions):
        """Make ensemble predictions"""
        if not self.is_loaded:
            self.load_model()
        
        try:
            # Create advanced features
            advanced_features = self.create_advanced_features(component_predictions)
            
            # Get predictions from all meta-classifiers
            meta_predictions = []
            for meta_classifier in self.meta_classifiers:
                if hasattr(meta_classifier, 'predict_proba'):
                    pred = meta_classifier.predict_proba(advanced_features)[:, 1]
                else:
                    pred = meta_classifier.predict(advanced_features).astype(float)
                meta_predictions.append(pred)
            
            # Final ensemble
            final_prediction = np.mean(meta_predictions, axis=0)
            
            return final_prediction
            
        except Exception as e:
            print(f"Error in ensemble prediction: {e}")
            return np.mean(component_predictions, axis=0)

In [10]:
class StatisticalFeatureExtractor:
    """Statistical feature extraction for inference"""
    
    def __init__(self):
        self.feature_names = []
    
    def extract_features(self, texts):
        """Extract statistical features"""
        features = []
        
        for text in texts:
            try:
                text_features = {}
                
                # Basic statistics
                text_features['length'] = len(text)
                words = text.split()
                text_features['word_count'] = len(words)
                sentences = re.split(r'[.!?]+', text)
                text_features['sentence_count'] = len([s for s in sentences if s.strip()])
                
                # Word-level statistics
                if words:
                    text_features['avg_word_length'] = np.mean([len(word) for word in words])
                    text_features['max_word_length'] = max(len(word) for word in words)
                    text_features['unique_word_ratio'] = len(set(words)) / len(words)
                else:
                    text_features['avg_word_length'] = 0
                    text_features['max_word_length'] = 0
                    text_features['unique_word_ratio'] = 0
                
                # Character statistics
                text_features['punct_ratio'] = len(re.findall(r'[^\w\s]', text)) / max(len(text), 1)
                text_features['caps_ratio'] = len(re.findall(r'[A-Z]', text)) / max(len(text), 1)
                text_features['digit_ratio'] = len(re.findall(r'\d', text)) / max(len(text), 1)
                
                # Sentence statistics
                if text_features['sentence_count'] > 0:
                    text_features['avg_sentence_length'] = text_features['word_count'] / text_features['sentence_count']
                else:
                    text_features['avg_sentence_length'] = 0
                
                # Additional features
                text_features['question_count'] = text.count('?')
                text_features['exclamation_count'] = text.count('!')
                text_features['comma_count'] = text.count(',')
                
                complex_words = [word for word in words if len(word) > 6]
                text_features['complex_word_ratio'] = len(complex_words) / max(len(words), 1)
                
                features.append(text_features)
                
            except Exception as e:
                features.append({
                    'length': 0, 'word_count': 0, 'sentence_count': 0,
                    'avg_word_length': 0, 'max_word_length': 0, 'unique_word_ratio': 0,
                    'punct_ratio': 0, 'caps_ratio': 0, 'digit_ratio': 0,
                    'avg_sentence_length': 0, 'question_count': 0,
                    'exclamation_count': 0, 'comma_count': 0, 'complex_word_ratio': 0
                })
        
        df = pd.DataFrame(features)
        self.feature_names = df.columns.tolist()
        return df

In [11]:
class InferenceSolution:
    """Main inference solution class"""
    
    def __init__(self):
        self.deberta_inference = None
        self.zero_shot_inference = None
        self.tfidf_inference = None
        self.ensemble_inference = None
        
    def load_all_models(self):
        """Load all trained models"""
        print("Loading all trained models...")
        
        try:
            # Load DeBERTa
            deberta_path = os.path.join(InferenceConfig.MODEL_SAVE_DIR, "deberta")
            if os.path.exists(deberta_path):
                self.deberta_inference = DeBERTaInference(deberta_path)
            
            # Load Zero-shot
            zero_shot_path = os.path.join(InferenceConfig.MODEL_SAVE_DIR, "zero_shot")
            if os.path.exists(zero_shot_path):
                self.zero_shot_inference = ZeroShotInference(zero_shot_path)
            
            # Load TF-IDF
            tfidf_path = os.path.join(InferenceConfig.MODEL_SAVE_DIR, "tfidf")
            if os.path.exists(tfidf_path):
                self.tfidf_inference = TFIDFInference(tfidf_path)
            
            # Load Ensemble
            ensemble_path = os.path.join(InferenceConfig.MODEL_SAVE_DIR, "ensemble")
            if os.path.exists(ensemble_path):
                self.ensemble_inference = EnsembleInference(ensemble_path)
            
            print("All models loaded successfully")
            
        except Exception as e:
            print(f"Error loading models: {e}")
            raise
    
    def load_test_data(self):
        """Load test data"""
        try:
            test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
            print(f"Test samples: {len(test_df)}")
            return test_df
        except Exception as e:
            print(f"Error loading test data: {e}")
            raise
    
    def preprocess_texts(self, texts):
        """Preprocess test texts"""
        processed_texts = []
        for text in texts:
            text = str(text).strip()
            text = re.sub(r'\s+', ' ', text)
            processed_texts.append(text)
        return processed_texts
    
    def make_predictions(self, test_df):
        """Make final predictions using all models"""
        print("Making predictions on test data...")
        
        try:
            texts = self.preprocess_texts(test_df['text'].values)
            
            # Get predictions from each model
            component_predictions = []
            
            # DeBERTa predictions
            if self.deberta_inference:
                print("Getting DeBERTa predictions...")
                deberta_preds = self.deberta_inference.predict(texts)
                component_predictions.append(deberta_preds)
            
            # TF-IDF predictions
            if self.tfidf_inference:
                print("Getting TF-IDF predictions...")
                tfidf_preds = self.tfidf_inference.predict(texts)
                component_predictions.append(tfidf_preds)
            
            # Zero-shot predictions (limit to reasonable number for speed)
            if self.zero_shot_inference:
                print("Getting zero-shot predictions...")
                zero_shot_sample_size = min(len(texts), 500)
                zero_shot_texts = texts[:zero_shot_sample_size]
                zero_shot_preds = self.zero_shot_inference.predict(zero_shot_texts)
                
                # Pad if needed
                if len(zero_shot_preds) < len(texts):
                    zero_shot_preds = np.pad(
                        zero_shot_preds,
                        (0, len(texts) - len(zero_shot_preds)),
                        mode='constant',
                        constant_values=0.5
                    )
                component_predictions.append(zero_shot_preds)
            
            # Ensemble prediction
            if self.ensemble_inference and len(component_predictions) >= 2:
                print("Getting ensemble predictions...")
                final_predictions = self.ensemble_inference.predict(component_predictions)
            elif component_predictions:
                # Simple average if no ensemble model
                print("Using simple average of component predictions...")
                final_predictions = np.mean(component_predictions, axis=0)
            else:
                print("No models available, using default predictions...")
                final_predictions = np.full(len(texts), 0.5)
            
            return final_predictions
            
        except Exception as e:
            print(f"Error making predictions: {e}")
            return np.full(len(test_df), 0.5)
    
    def create_submission(self, test_df, predictions):
        """Create submission file"""
        try:
            submission_df = pd.DataFrame({
                'id': test_df['id'],
                'generated': predictions
            })
            
            submission_df.to_csv('submission.csv', index=False)
            print(f"Submission saved with {len(submission_df)} predictions")
            print(f"Prediction statistics:")
            print(f"  Mean: {predictions.mean():.4f}")
            print(f"  Std: {predictions.std():.4f}")
            print(f"  Min: {predictions.min():.4f}")
            print(f"  Max: {predictions.max():.4f}")
            
            return submission_df
            
        except Exception as e:
            print(f"Error creating submission: {e}")
            raise
    
    def run_inference(self):
        """Run complete inference pipeline"""
        print("=" * 60)
        print("LLM AI Detection Competition - INFERENCE PHASE")
        print("=" * 60)
        print(f"Running on: {InferenceConfig.DEVICE}")
        print(f"Model directory: {InferenceConfig.MODEL_SAVE_DIR}")
        print("=" * 60)
        
        start_time = time.time()
        
        try:
            # Load models
            self.load_all_models()
            
            # Load test data
            test_df = self.load_test_data()
            
            # Make predictions
            predictions = self.make_predictions(test_df)
            
            # Create submission
            submission_df = self.create_submission(test_df, predictions)
            
            total_time = time.time() - start_time
            
            print("=" * 60)
            print("INFERENCE COMPLETED!")
            print("=" * 60)
            print(f"Total time: {total_time:.2f}s")
            print(f"Processed {len(test_df)} samples")
            print("Submission file: submission.csv")
            print("=" * 60)
            
            return submission_df
            
        except Exception as e:
            print(f"Error in inference: {e}")
            raise

In [12]:
# Main execution - INFERENCE ONLY
if __name__ == "__main__":
    # Initialize and run inference
    solution = InferenceSolution()
    submission = solution.run_inference()
    
    print("\n" + "="*60)
    print("INFERENCE PHASE COMPLETED!")
    print("Submission file created successfully.")
    print("="*60)

LLM AI Detection Competition - INFERENCE PHASE
Running on: cuda
Model directory: /kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/
Loading all trained models...
All models loaded successfully
Test samples: 3
Making predictions on test data...
Getting DeBERTa predictions...
Loading DeBERTa model from /kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/deberta


2025-07-18 15:57:33.093600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752854253.268136      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752854253.326558      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model trained for 12 epochs
DeBERTa model loaded successfully
Making DeBERTa predictions on 3 samples...
Getting TF-IDF predictions...
Loading TF-IDF model from /kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/tfidf
TF-IDF model loaded successfully
Making TF-IDF predictions on 3 samples...
Getting zero-shot predictions...
Loading zero-shot model from /kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/zero_shot


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Zero-shot model loaded successfully
Making zero-shot predictions on 3 samples...
Processing text 1/3
Getting ensemble predictions...
Loading ensemble model from /kaggle/input/k/modelbuilderpro/llm-detect-ai-generated-text/saved_models/ensemble
Ensemble model loaded successfully
Submission saved with 3 predictions
Prediction statistics:
  Mean: 0.2943
  Std: 0.0003
  Min: 0.2940
  Max: 0.2947
INFERENCE COMPLETED!
Total time: 36.21s
Processed 3 samples
Submission file: submission.csv

INFERENCE PHASE COMPLETED!
Submission file created successfully.
